In [1]:
from ultralytics import YOLO
import cv2
import os
import numpy as np

# 1. 对于YOLOv8的极大值抑制

In [2]:
# NMS function
def non_max_suppression(boxes, conf_thresh=0.5, iou_thresh=0.4):
    # Filter out boxes with low confidence scores
    boxes = [box for box in boxes if box[4] > conf_thresh]

    if len(boxes) == 0:
        return []

    boxes = np.array(boxes)
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]
    scores = boxes[:, 4]

    # Calculate area of the boxes
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
        inter = w * h
        iou = inter / (areas[i] + areas[order[1:]] - inter)

        inds = np.where(iou <= iou_thresh)[0]
        order = order[inds + 1]

    return boxes[keep]

# 2. 主要的测试程序部分

In [4]:
# MOT15数据集类别
MOT15_BASE_DIR = r'C:\Users\data\Documents\MOT15\MOT15\train\ADL-Rundle-8\img1'

# 加载YOLO模型
yolo = YOLO('yolov8s.pt')

# Load images from the dataset
images = [img_path for img_path in os.listdir(MOT15_BASE_DIR) if img_path.endswith('.jpg')]

# Define COCO classes
CLASSES = ['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
              'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
              'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
              'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
              'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
              'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa',
              'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard',
              'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
              'teddy bear', 'hair drier', 'toothbrush']

# Show testing images
for img_path in images:

    # Read image
    img = cv2.imread(os.path.join(MOT15_BASE_DIR, img_path))
   
    # Perform detection, and silence the output
    detections = yolo(img, verbose=False)

    # Process results list
    for result in detections:
        boxes = result.boxes  # Boxes object for bounding box outputs
        boxes_data = []

        for box in boxes:
            data = box.data.cpu().numpy().reshape(-1)
            x1 = int(data[0])
            y1 = int(data[1])
            x2 = int(data[2])
            y2 = int(data[3])
            conf = float(data[4])
            cls = int(data[5])
            boxes_data.append([x1, y1, x2, y2, conf, cls])

        # NMS
        nms_boxes = non_max_suppression(boxes_data, conf_thresh=0.5, iou_thresh=0.5)

        # Draw bounding boxes
        for box in nms_boxes:
            x1, y1, x2, y2, conf, cls = box

            # Convert to integers
            x1, y1, x2, y2, cls = int(x1), int(y1), int(x2), int(y2), int(cls)

            # Draw bounding box
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Draw confidence
            cv2.putText(img, f'{conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Draw class
            cv2.putText(img, f'{CLASSES[cls]}', (x1, y1 - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Resize the image to 800x600
    img = cv2.resize(img, (800, 600))

    # Display the image
    cv2.imshow('Image', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()